In [0]:
#Upload kaggle.json
from google.colab import files
files.upload()

In [0]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
#Copy API from kaggle page
!kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge


In [11]:
!ls


dataset  example_submission.csv  fer2013.bib  fer2013.tar.gz  README


In [0]:
!mkdir dataset
import tarfile
tar = tarfile.open("./fer2013.tar.gz")
tar.extractall('./dataset')
tar.close()

In [13]:
%cd ./dataset/fer2013
!mv fer2013.csv /content/

/content/dataset/fer2013/dataset/fer2013


In [14]:
%cd /content

/content


In [15]:
!ls


dataset			fer2013.csv	kaggle.json
example_submission.csv	fer2013.tar.gz	sample_data


In [16]:
!rm -r dataset
!rm example_submission.csv
!rm fer2013.tar.gz
!rm -r sample_data
!rm -r fer2013

rm: cannot remove 'fer2013': No such file or directory


In [17]:
!ls
!pwd

fer2013.csv  kaggle.json
/content


In [18]:
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize

img_width = 48
img_height = 48
dataset = pd.read_csv('./fer2013.csv')


Using TensorFlow backend.


In [0]:
Z = dataset.iloc[:,2].values

In [20]:
temp = 0
for x in Z:
  if str(x) != 'Training':
    break
  temp = temp+1

print(temp)

28709


In [0]:
X = dataset.iloc[:, 1].values
y = dataset.iloc[:, 0].values

images = np.empty((len(X), img_height, img_width, 3))
i=0
for pixel_string in X:
  temp = [float(pixel) for pixel in pixel_string.split(' ')]
  temp = np.asarray(temp).reshape(img_height, img_width)
  temp = resize(temp, (img_height, img_width), order=3, mode='constant')

  channel = np.empty((img_height, img_width, 3))
  channel[:, :, 0] = temp
  channel[:, :, 1] = temp
  channel[:, :, 2] = temp

  images[i, :, :, :] = channel
  i = i + 1

images /= 255.0
labels = keras.utils.to_categorical(y, 7)   

crossvalidation_set = images[28709:,:,:,:]
images = images[0:28909,:,:,:]
 


In [0]:
cross_label = labels[28709:,:]
labels = labels[0:28909,:]

In [0]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPooling2D,MaxPooling1D
from keras.layers import Activation,Dropout,Flatten,BatchNormalization
from keras import backend as K
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard

In [0]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(
        rescale = 1./1)

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(images)

# fits the model on batches with real-time data augmentation:
training_datagen = datagen.flow(images, labels, batch_size=32)

validation_datagen = test_datagen.flow(crossvalidation_set,cross_label,batch_size = 32)

In [37]:
input_shape = (48, 48, 3)
batch_size = 64
epochs = 40
verbose = 1

model = Sequential()
model.add(Conv2D(
    32,
    kernel_size=(3, 3),
    activation='relu',
    input_shape=input_shape
))

model.add(Dropout(0.35))

model.add(Conv2D(
    64,
    (3, 3),
    activation='relu'
))

model.add(Dropout(0.25))

model.add(MaxPooling2D(pool_size=(2, 2),padding = 'same'))

model.add(Conv2D(
    128,
    (3, 3),
    activation='relu'
))

model.add(Conv2D(
    256,
    (3, 3),
    activation='relu'
))

model.add(MaxPooling2D(pool_size=(3, 3)))

model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.35))
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.35))
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.35))
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.35))
model.add(Dense(7, activation='softmax'))

model.compile(
    optimizer='adam',
    metrics=['accuracy'],
    loss='categorical_crossentropy'
)

model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 46, 46, 32)        896       
_________________________________________________________________
dropout_15 (Dropout)         (None, 46, 46, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 44, 44, 64)        18496     
_________________________________________________________________
dropout_16 (Dropout)         (None, 44, 44, 64)        0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 20, 20, 128)       73856     
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 18, 18, 256)       295168    
__________

In [38]:
history = model.fit(
    images, labels,
    validation_data = (crossvalidation_set,cross_label),
    batch_size=batch_size,
    verbose=verbose,
    epochs=epochs
)

Train on 28909 samples, validate on 7178 samples
Epoch 1/40
28909/28909 [==============================] - 47s 2ms/step - loss: 1.9123 - acc: 0.2286 - val_loss: 2.1348 - val_acc: 0.1718
Epoch 2/40
28909/28909 [==============================] - 45s 2ms/step - loss: 1.7112 - acc: 0.3169 - val_loss: 1.7052 - val_acc: 0.3222
Epoch 3/40
28909/28909 [==============================] - 45s 2ms/step - loss: 1.5425 - acc: 0.3981 - val_loss: 1.5925 - val_acc: 0.3750
Epoch 4/40
28909/28909 [==============================] - 45s 2ms/step - loss: 1.4152 - acc: 0.4549 - val_loss: 1.4735 - val_acc: 0.4344
Epoch 5/40
28909/28909 [==============================] - 45s 2ms/step - loss: 1.3320 - acc: 0.4902 - val_loss: 1.3820 - val_acc: 0.4868
Epoch 6/40
28909/28909 [==============================] - 45s 2ms/step - loss: 1.2577 - acc: 0.5237 - val_loss: 1.3267 - val_acc: 0.4993
Epoch 7/40
28909/28909 [==============================] - 45s 2ms/step - loss: 1.2010 - acc: 0.5429 - val_loss: 1.2761 - val_acc: